# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 9:25AM,261478,10,SO000340,Alliance Pharma LTD,Released
1,Apr 20 2023 9:25AM,261478,10,SO000327,Alliance Pharma LTD,Released
2,Apr 20 2023 9:25AM,261478,10,SO000330,Alliance Pharma LTD,Released
3,Apr 20 2023 9:08AM,261473,10,HH-41415,Hush Hush,Released
4,Apr 20 2023 9:08AM,261473,10,HH-41416,Hush Hush,Released
5,Apr 20 2023 9:08AM,261473,10,HH-41417,Hush Hush,Released
6,Apr 20 2023 9:08AM,261473,10,HH-41418,Hush Hush,Released
7,Apr 20 2023 9:08AM,261473,10,HH-41419,Hush Hush,Released
8,Apr 20 2023 9:08AM,261473,10,HH-41420,Hush Hush,Released
9,Apr 20 2023 9:08AM,261473,10,HH-41421,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,261472,Released,1
20,261473,Released,19
21,261474,Released,3
22,261475,Executing,1
23,261478,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261472,NaN,NaN,1.0
261473,NaN,NaN,19.0
261474,NaN,NaN,3.0
261475,NaN,1.0,NaN
261478,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261398,28.0,14.0,0.0
261405,0.0,0.0,2.0
261406,0.0,1.0,0.0
261410,2.0,2.0,5.0
261432,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261398,28,14,0
261405,0,0,2
261406,0,1,0
261410,2,2,5
261432,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261398,28,14,0
1,261405,0,0,2
2,261406,0,1,0
3,261410,2,2,5
4,261432,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261398,28,14,
1,261405,,,2
2,261406,,1,
3,261410,2,2,5
4,261432,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 9:25AM,261478,10,Alliance Pharma LTD
3,Apr 20 2023 9:08AM,261473,10,Hush Hush
22,Apr 20 2023 9:07AM,261475,10,"Senseonics, Incorporated"
23,Apr 20 2023 9:07AM,261474,10,"Digital Brands, LLC"
26,Apr 20 2023 8:56AM,261472,10,Hush Hush
27,Apr 20 2023 8:54AM,261471,10,Emerginnova
29,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc
33,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc."
34,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc."
38,Apr 20 2023 8:46AM,261466,19,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 20 2023 9:25AM,261478,10,Alliance Pharma LTD,,,3
1,Apr 20 2023 9:08AM,261473,10,Hush Hush,,,19
2,Apr 20 2023 9:07AM,261475,10,"Senseonics, Incorporated",,1,
3,Apr 20 2023 9:07AM,261474,10,"Digital Brands, LLC",,,3
4,Apr 20 2023 8:56AM,261472,10,Hush Hush,,,1
5,Apr 20 2023 8:54AM,261471,10,Emerginnova,,,2
6,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,,,4
7,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",,,1
8,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",,,4
9,Apr 20 2023 8:46AM,261466,19,"Methapharm, Inc.",,2,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 9:25AM,261478,10,Alliance Pharma LTD,3,,
1,Apr 20 2023 9:08AM,261473,10,Hush Hush,19,,
2,Apr 20 2023 9:07AM,261475,10,"Senseonics, Incorporated",,1,
3,Apr 20 2023 9:07AM,261474,10,"Digital Brands, LLC",3,,
4,Apr 20 2023 8:56AM,261472,10,Hush Hush,1,,
5,Apr 20 2023 8:54AM,261471,10,Emerginnova,2,,
6,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,4,,
7,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",1,,
8,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",4,,
9,Apr 20 2023 8:46AM,261466,19,"Methapharm, Inc.",2,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 9:25AM,261478,10,Alliance Pharma LTD,3,,
1,Apr 20 2023 9:08AM,261473,10,Hush Hush,19,,
2,Apr 20 2023 9:07AM,261475,10,"Senseonics, Incorporated",,1,
3,Apr 20 2023 9:07AM,261474,10,"Digital Brands, LLC",3,,
4,Apr 20 2023 8:56AM,261472,10,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 9:25AM,261478,10,Alliance Pharma LTD,3.0,NaN,NaN
1,Apr 20 2023 9:08AM,261473,10,Hush Hush,19.0,NaN,NaN
2,Apr 20 2023 9:07AM,261475,10,"Senseonics, Incorporated",NaN,1.0,NaN
3,Apr 20 2023 9:07AM,261474,10,"Digital Brands, LLC",3.0,NaN,NaN
4,Apr 20 2023 8:56AM,261472,10,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 9:25AM,261478,10,Alliance Pharma LTD,3.0,0.0,0.0
1,Apr 20 2023 9:08AM,261473,10,Hush Hush,19.0,0.0,0.0
2,Apr 20 2023 9:07AM,261475,10,"Senseonics, Incorporated",0.0,1.0,0.0
3,Apr 20 2023 9:07AM,261474,10,"Digital Brands, LLC",3.0,0.0,0.0
4,Apr 20 2023 8:56AM,261472,10,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2353248,37.0,1.0,0.0
19,2353004,101.0,16.0,28.0
20,522816,5.0,3.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2353248,37.0,1.0,0.0
1,19,2353004,101.0,16.0,28.0
2,20,522816,5.0,3.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,37.0,1.0,0.0
1,19,101.0,16.0,28.0
2,20,5.0,3.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,19,Released,101.0
2,20,Released,5.0
3,10,Executing,1.0
4,19,Executing,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Completed,0.0,28.0,2.0
Executing,1.0,16.0,3.0
Released,37.0,101.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Completed,0.0,28.0,2.0
1,Executing,1.0,16.0,3.0
2,Released,37.0,101.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Completed,0.0,28.0,2.0
1,Executing,1.0,16.0,3.0
2,Released,37.0,101.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()